In [1]:
import fasttext
import numpy as np
import json
import random
import datetime
import os

Via https://fasttext.cc

```
wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.it.300.bin.gz
wget https://dl.fbaipublicfiles.com/fasttext/vectors-wiki/wiki.it.zip
```

In [2]:
wikimodel = fasttext.FastText.load_model("data/wiki.it/wiki.it.bin")

ccmodel = fasttext.FastText.load_model("data/cc.it.300.bin")

In [3]:
def cosine_distance(first, second, model, decimals=2):
    exact_distance = (np.dot(model[first], model[second])
            /(np.linalg.norm(model[first])*np.linalg.norm(model[second]))
           )
    truncated_distance = np.round(float(exact_distance*100),decimals)
    return truncated_distance

## Extract Apple dictionary

https://gist.github.com/josephg/5e134adf70760ee7e49d

In [4]:
# Thanks to commenters for providing the base of this much nicer implementation!
# Save and run with $ python 0dedict.py
# You may need to hunt down the dictionary files yourself and change the awful path string below.
# This works for me on MacOS 10.14 Mohave

from struct import unpack
from zlib import decompress
import re
# filename = '/System/Library/Assets/com_apple_MobileAsset_DictionaryServices_dictionaryOSX/9f5862030e8f00af171924ebbc23ebfd6e91af78.asset/AssetData/Oxford Dictionary of English.dictionary/Contents/Resources/Body.data'
filename = "/System/Library/AssetsV2/com_apple_MobileAsset_DictionaryServices_dictionaryOSX/bad4a5c5bf10a2dd135a07d7ad7b28e23d0aa7dd.asset/AssetData/Italian.dictionary/Contents/Resources/Body.data"

f = open(filename, 'rb')

def gen_entry():
    f.seek(0x40)
    limit = 0x40 + unpack('i', f.read(4))[0]
    f.seek(0x60)
    while f.tell()<limit:
        sz, = unpack('i', f.read(4))
        buf = decompress(f.read(sz)[8:])

        pos = 0
        while pos < len(buf):
            chunksize, = unpack('i', buf[pos:pos+4])
            pos += 4

            entry = buf[pos:pos+chunksize]
            title = re.search('d:title="(.*?)"', entry.decode('utf-8')).group(1)
            yield title, entry

            pos += chunksize

wordlist = []
for word, definition in gen_entry():
    wordlist.append(word)

a
a
a-
a(d)-
a'
ab-
abaca
abacà
abaco
abadessa
abadia
abalietà
ab antiquo
abarico
abasia
abate
abatino
abat-jour
abaton
abaya
abazia
abaziale
abbacare
abbacchiamento
abbacchiare
abbacchiato
abbacchiatura
abbacchio
abbacinamento
abbacinante
abbacinare
abbaco
abbadare
abbadessa
abbadia
abbagliamento
abbagliante
abbagliare
abbaglio
abbaiare
abbaiata
abbaino
abbaio
abbaio
abballare
abballottare
abbambinare
abbambolato
abbancare
abbandonare
abbandonato
abbandonico
abbandonismo
abbandono
abbarbagliamento
abbarbagliare
abbarbaglio
abbarbaglio
abbarbicamento
abbarbicante
abbarbicare
abbarcare
abbarcare
abbarcatura
abbarrare
abbaruffare
abbaruffio
abbassabile
abbassalingua
abbassamento
abbassare
abbassata
abbasso
abbastanza
abbate
abbattere
abbattibile
abbattifieno
abbattimento
abbattitore
abbattuta
abbattuto
abbatuffolare
abbazia
abbaziale
abbecedario
abbellare
abbellimento
abbellire
abbellitura
abbenché
abbeveraggio
abbeverare
abbeverata
abbeveratoio
abbiadare
abbiatico
abbicare
abbicatura
ab

adenovirosi
adenovirus
adepto
adequare
aderbare
aderente
aderenza
adergere
aderimento
aderire
aderizzare
aderizzazione
adescabile
adescamento
adescare
adescatore
adesione
adesività
adesivo
adeso
adespoto
adesso
ad hoc
ad honorem
adiabatico
adiacente
adiacenza
adiaforia
adiaforo
Adiantacee
adianto
adiastematico
adiastolia
adiatermanità
adiatermano
adibire
adiettivo
adiga
adimare
adimensionale
adinamia
adinato
ad interim
adipato
adipe
adipectomia
adipico
adiponectina
adiposi
adiposità
adiposo
adipsia
adiramento
adirare
adirato
adire
adito
adito
adiutore
adiutorio
adiuvante
adiuvare
adizione
adizzare
ad libitum
admin
adnominale
ad nutum
-ado
adobe
adocchiamento
adocchiare
adolescente
adolescenza
adolescenziale
adombrabile
adombramento
adombrare
adonare
Adone
adonestare
adonide
adonio
adontamento
adontare
adoperabile
adoperare
adoprare
adorabile
adorabilità
adorale
adorare
adoratore
adorazione
adorezzare
adornamento
adornare
adornatore
adorno
adottabile
adottabilità
adottamento
adottando
a

algoritmico
algoritmo
algoso
-ali
aliante
aliantista
aliare
alias
alibi
alicante
alice
aliciclico
alicorno
alidada
alido
alidore
alienabile
alienabilità
alienamento
alienante
alienare
alienato
alienatore
alienazione
alienia
alienista
alienità
alieno
alieutica
alifatico
aliforme
aliga
aligero
alighiero
aligusta
alimentare
alimentare
alimentario
alimentarista
alimentatore
alimentazione
alimento
alimo
alimonia
alinea
aliosso
aliotide
alipede
aliquota
aliscafo
aliseo
alisma
Alismatacee
alisso
alitare
alite
alito
alitosi
aliturgico
alivolo
alizarina
alla
allacciamento
allacciare
allacciatura
allaccio
allagamento
allagare
allalì
allampanare
allampanato
allantiasi
allantoide
allantoideo
allappante
allappare
allargamento
allargare
allargata
allargatoio
allargatore
allargatrice
allargatubi
allargatura
allarmante
allarmare
allarmato
allarme
allarmismo
allarmista
allarmistico
allascare
allativo
allato
allattamento
allattare
allattatrice
allattatura
alle
allea
alleanza
alleare
alleato
allegabile
a

anabbagliante
anabiosi
anabolico
anabolismo
anabolizzante
Anacardiacee
anacardio
anace
anacenosi
anaciato
anaciclico
anacino
anacio
anaclasi
anaclastico
anacloridria
anacoluto
anaconda
anacoresi
anacoreta
anacoretico
anacoretismo
anacreonteo
anacreontica
anacreontico
anacronismo
anacronistico
anacrusi
anadenia
anadiomene
anadiplosi
anadromo
anaelettrico
anaerobico
anaerobio
anaerobiosi
anafase
anaffettività
anaffettivo
anafilassi
anafilattico
anafonesi
anafora
anaforesi
anaforico
anafrodisia
anafrodisiaco
anagallide
anagenesi
anagiride
anaglifo
anagliptico
anaglipto
anaglittica
anaglittico
anagogia
anagogico
anagrafe
anagrafico
anagramma
anagrammare
anagrammatico
anagrammato
anagrammista
analcolico
anale
analemma
analessi
analettico
analfabeta
analfabetico
analfabetismo
analfabeto
analgesia
analgesico
analisi
analista
analita
analitica
analiticamente
analiticità
analitico
analizzabile
analizzare
analizzatore
anallergico
analogia
analogico
analogismo
analogista
analogo
anamnesi
anamnest

aplestia
aplisia
Aplisidi
aplo-
aplodonte
aplografia
aploide
aploidia
aplologia
aplologico
aplomb
aplostele
aplotipo
aplustre
apnea
apneista
apneusi
apnoico
apo-
apocalisse
apocalittico
apocatastasi
Apocinacee
apocino
apocizio
apocopare
apocope
apocrifo
apocrino
apocromatico
apode
Apodi
Apodidi
Apodiformi
apodissi
apoditerio
apodittica
apoditticità
apodittico
apodo
apodosi
apofantico
apofatico
apofisario
apofisi
apofita
apofonia
apofonico
apoftegma
apogeo
apografo
apolare
apolide
apolidia
apolitia
apoliticità
apolitico
apollineo
apollo
apologeta
apologetica
apologetico
apologia
apologista
apologistico
apologizzare
apologo
apomorfo
aponense
aponeurosi
aponeurotico
aponevrosi
aponevrotico
aponia
apoplessia
apoplettico
apoptosi
aporema
aporetica
aporetico
aporia
aporia
aposematico
aposematismo
aposepalo
aposiopesi
apostasia
apostata
apostatare
apostema
a posteriori
apostolato
apostolicità
apostolico
apostolo
apostrofare
apostrofare
apostrofe
apostrofo
apoteca
apotecio
apotema
apoteosi
apo

...

trebbiatrice
trebbiatura
trebbio
trebbo
trebisonda
treccia
trecciare
trecentesco
trecentista
trecentistico
trecento
tredecimo
tredicesima
tredicesimo
tredici
tredicista
trefolo
trefoni
tregenda
treggea
treggia
tregua
trekking
tremacchio
tremacore
tremacuore
tremaglio
tremante
tremarcto
tremare
tremarella
Trematodi
tremebondo
tremendismo
tremendo
trementina
tremerella
tremila
tremisse
tremito
tremolamento
tremolante
tremolare
tremolina
tremolio
tremolo
tremore
tremulo
trenaggio
trench
trend
trending topic
trend setter
trendy
trenetta
trenino
treno
treno
trenodia
trenta
trentaduesimo
trentatré
trentennale
trentennio
trentina
trentino
treonina
trepak
trepang
trepestare
trepidante
trepidanza
trepidare
trepidazione
trepidezza
trepido
treponema
Treponematacee
treponti
treppiede
treppiedi
trepponti
trequarti
trequartista
treruote
tresca
trescare
Treschiornitidi
trescone
tresette
trespide
trespolo
tressette
trestizia
treto
treviere
trevigiano
trevira
trevisano
trevo
trezza
tri-
tria
triaca
tri

uggire
-uglio
ugna
ugnare
ugnata
ugnatura
ugnella
ugnello
ugnere
ugnetto
ugola
ugonotto
ugrico
ugro-finnico
uguagliamento
uguaglianza
uguagliare
uguagliatore
uguale
ugualità
ugualitario
ugualitarismo
ugualizzante
ugualizzare
ugualizzatore
ugualizzazione
uguanno
uh
uhi
uhm
uiguro
uistitì
uitlanders
ukase
ukulele
ulà
ulama
ulano
ulca
ulcera
ulcerare
ulcerativo
ulcerato
ulcerazione
ulcere
ulcerogeno
ulceroso
ulema
uliganismo
uligano
uligine
uliginoso
ulisside
ulissismo
uliva
ulivella
ulivista
ulivo
Ulmacee
ulmico
ulna
ulnare
ulotrice
ulotrico
ulteriore
ulteriormente
ultima
ultimabile
ultimamente
ultimare
ultimativo
ultimatum
ultimazione
ultimissima
ultimo
ultimogenito
ulto
ultore
ultra-
ultrà
ultrabook
ultracentenario
ultracentrifuga
ultracentrifugazione
ultracompatto
ultracondensatore
ultracorto
ultracustica
ultracustico
ultrafacile
ultrafemminile
ultrafiltrante
ultrafiltrazione
ultrafiltro
ultraforming
ultraleggero
ultramarino
ultramicrofotografia
ultramicrometro
ultramicroscopia
ultram

venditore
-vendolo
venduto
veneficio
venefico
veneno
venerabile
venerabilità
venerando
venerare
veneratore
venerazione
venerdì
Venere
venereo
venereologia
Veneridi
venetico
veneto
veneziana
veneziano
venezolano
venezuelano
vengiare
venia
veniale
venialità
veniente
venire
venosino
venosità
venoso
ventaglia
ventaglina
ventaglio
ventare
ventarola
ventata
ventatura
ventennale
ventenne
ventennio
ventesimo
venti
venticinque
ventidue
ventiera
ventilabro
ventilare
ventilatore
ventilatorista
ventilazione
ventimila
ventina
ventino
ventiquattresimo
ventiquattro
ventiquattrore
ventiquattr'ore
ventisettana
ventisette
ventitré
vento
ventola
ventolare
ventolatura
ventosa
ventosare
ventosità
ventoso
ventoso
ventraia
ventrale
ventralista
ventre
ventresca
ventricina
ventricolare
ventricolo
ventricoso
ventriera
ventriglio
ventriloquia
ventriloquio
ventriloquo
ventrino
ventuno
ventura
venture capital
venturiere
venturiero
venturimetro
venturina
venturo
venturone
venturoso
venula
venusiano
venustà
venusto


vivinatalità
viviparità
viviparo
vivisettore
vivisettorio
vivisezionare
vivisezione
vivo
vivucchiare
vivuola
viziare
viziato
viziatura
vizietto
vizio
viziosità
vizioso
vizzo
vlog
vlogger
vocabolariesco
vocabolario
vocabolarista
vocabolaristico
vocabolarizzare
vocabolarizzazione
vocabolista
vocabolo
vocabulista
vocale
vocale
vocalico
vocalismo
vocalist
vocalista
vocalità
vocalizzare
vocalizzazione
vocalizzo
vocare
vocativo
vocazionale
vocazione
voce
vocero
vocianesimo
vociano
vociare
vociferare
vociferatore
vociferazione
vocio
vocione
vocoder
vocoide
vodese
vodka
vodù
voga
voga
vogare
vogata
vogatore
voglia
vogliente
voglienza
voglio ma non posso
vogliosità
voglioso
voi
voialtri
voilà
voile
voivoda
volanda
volandiero
volandola
volano
volant
volante
volante
volantinaggio
volantinare
volantinatore
volantino
volantino
volapük
volare
volare
volata
volatica
volatico
volatile
volatilità
volatilizzare
volatilizzazione
volatio
volativo
volatizzare
volato
volatona
volatone
volatore
vol-au-vent
v

zolfino
zolfo
zolla
zollare
zolletta
zollosità
zolloso
zombare
zombata
zombatura
zombi
zombie
zompare
zompata
zompettare
zompo
zona
zonale
zonalità
zonarolo
zonato
zonatura
zonazione
zoneare
zoneggiare
zoning
zonizzare
zonizzazione
zonula
zonulare
zonzo
zoo
zoo-
zooantropologia
zoocenosi
zoocida
Zooclorelle
zoocoltura
zooconidio
zoocoria
zoocoro
zoocultura
zoofagia
zoofago
zoofenologia
zoofilia
zoofilo
Zooflagellati
zoofobia
zoofobo
zooforo
zoogamete
zoogamia
zoogamo
zoogenico
zoogeografia
zoogeografico
zooglea
zoognostica
zooiatra
zooiatria
zooiatrico
zooiatro
zooide
zoolatra
zoolatria
zoolatrico
zoolico
zoolito
zoologia
zoologico
zoologo
zoom
zoomafia
zoomare
zoomata
zoometria
zoomorfismo
zoomorfo
zoomorfosi
zoonimo
zoonomo
zoonosi
zoopaleontologia
zooparassita
zooplancton
zooprofilassi
zooprofilattico
zoopsia
zoosafari
zoosemiotica
zoospermio
zoospora
zoosterolo
zootecnia
zootecnico
zootomia
zootossina
Zooxantelle
zopolo
zoppaggine
zoppare
zoppeggiare
zoppia
zoppicamento
zoppicante


In [5]:
print(len(wordlist))

96859


# Parole italiane

```
git clone https://github.com/napolux/paroleitaliane.git

sudo apt-get install witalian

/usr/share/dict/italian
```

In [6]:
parole60  = "data/paroleitaliane/paroleitaliane/60000_parole_italiane.txt"
parole95  = "data/paroleitaliane/paroleitaliane/95000_parole_italiane_con_nomi_propri.txt"

parole280 = "data/paroleitaliane/paroleitaliane/280000_parole_italiane.txt"

paroleDebian = "data/italian"

# with open(paroleDebian, "r") as file:
#     words = [w.strip() for w in file.readlines() if "'" not in w]
    
words = [w for w in wordlist if len(w)>1 and '-' not in w and '(' not in w and ')' not in w and "'" not in w]

In [7]:
words

['abaca',
 'abacà',
 'abaco',
 'abadessa',
 'abadia',
 'abalietà',
 'ab antiquo',
 'abarico',
 'abasia',
 'abate',
 'abatino',
 'abaton',
 'abaya',
 'abazia',
 'abaziale',
 'abbacare',
 'abbacchiamento',
 'abbacchiare',
 'abbacchiato',
 'abbacchiatura',
 'abbacchio',
 'abbacinamento',
 'abbacinante',
 'abbacinare',
 'abbaco',
 'abbadare',
 'abbadessa',
 'abbadia',
 'abbagliamento',
 'abbagliante',
 'abbagliare',
 'abbaglio',
 'abbaiare',
 'abbaiata',
 'abbaino',
 'abbaio',
 'abbaio',
 'abballare',
 'abballottare',
 'abbambinare',
 'abbambolato',
 'abbancare',
 'abbandonare',
 'abbandonato',
 'abbandonico',
 'abbandonismo',
 'abbandono',
 'abbarbagliamento',
 'abbarbagliare',
 'abbarbaglio',
 'abbarbaglio',
 'abbarbicamento',
 'abbarbicante',
 'abbarbicare',
 'abbarcare',
 'abbarcare',
 'abbarcatura',
 'abbarrare',
 'abbaruffare',
 'abbaruffio',
 'abbassabile',
 'abbassalingua',
 'abbassamento',
 'abbassare',
 'abbassata',
 'abbasso',
 'abbastanza',
 'abbate',
 'abbattere',
 'abbattibil

In [10]:
len(words)

95264

In [11]:
with open("data/paroleitaliane/paroleitaliane/1000_parole_italiane_comuni.txt", "r") as file:
    common_words = [w.strip() for w in file.readlines()]
    
common_words = [w for w in common_words if len(w)>2]
len(common_words)

1132

In [12]:
def create_semantle_data(all_words, solution_words, model, date, puzzle_number, verbose=False):
    # Day to use for the solution
    isoday = date.isoformat()
    
    # Solution
    # Seed the random number generator
    random.seed(a=isoday, version=2)
    # Pick the word
    query = random.choice(solution_words)
    if verbose:
        print("La soluzione per il {} è \"{}\"".format(isoday, query))
        
    # Distanze
    distances_list = [(word, cosine_distance(query, word, model)) for word in set(all_words+solution_words) if len(word)>1]
    
    # Distanze nell'ordine giusto
    distances_list_sorted = sorted(distances_list, key=lambda x: x[1],reverse=True)
    
    # Database per Semantle
    
    word_database = {
        _tuple[0]:
        {
            "s": _tuple[1],
            "r": _ix,
        } for _ix, _tuple in enumerate(distances_list_sorted)
    }
    
    # Le parole più vicine
    closest_words_list = [
        {
            "w": _tuple[0],
            "s": _tuple[1],
            "r": _ix,
        } for _ix, _tuple in enumerate(distances_list_sorted[:1000])
    ]
    
    solution_word = query
    
    # Statistiche
    
    nearest_word_similarity = closest_words_list[1]["s"]
    tenth_nearest_word_similarity = closest_words_list[9]["s"]
    thousandth_nearest_word_similarity = closest_words_list[999]["s"]

    day_stats = {
        "puzzle_number": puzzle_number,
        "nearest_word_similarity": nearest_word_similarity,
        "tenth_nearest_word_similarity": tenth_nearest_word_similarity,
        "thousandth_nearest_word_similarity": thousandth_nearest_word_similarity,
    }

    semantle_data = {
        "word_database": word_database,
        "closest_words_list": closest_words_list,
        "solution_word": solution_word,
        "day_stats": day_stats,
    }
    return semantle_data

In [13]:
wikidata = create_semantle_data(
    all_words=words,
    solution_words=common_words,
    model=wikimodel,
    date=datetime.date.today()-datetime.timedelta(days=2),
    puzzle_number=-1,
    verbose=True
)

ccdata = create_semantle_data(
    all_words=words,
    solution_words=common_words,
    model=ccmodel,
    date=datetime.date.today()-datetime.timedelta(days=2),
    puzzle_number=-1,
    verbose=True
)

La soluzione per il 2022-09-11 è "nobile"
La soluzione per il 2022-09-11 è "nobile"


In [14]:
print([w['w'] for w in wikidata['closest_words_list'][:50]])
# print(wikidata['word_database']['re'])

['nobile', 'nobildonna', 'nobiluomo', 'nobilesco', 'nobiliare', 'nobilume', 'casata', 'conte', 'contessa', 'casato', 'nobilomo', 'nobiltà', 'nobilitato', 'aristocratico', 'gentildonna', 'nobiliario', 'marchese', 'duca', 'rampollo', 'marchesa', 'nobilitare', 'condottiero', 'viscontessa', 'duchessa', 'possidente', 'baronessa', 'terriero', 'discendente', 'nobilitade', 'ultimogenito', 'principessa', 'salottiero', 'morganatico', 'baronale', 'barone', 'benestante', 'quartogenito', 'bisnipote', 'comitale', 'secondogenito', 'paggio', 'genealogista', 'baronesco', 'cortigiano', 'sestogenito', 'mecenate', 'possidenza', 'sovrana', 'aristocrazia', 'quintogenito']


In [15]:
print([w['w'] for w in ccdata['closest_words_list'][:50]])
# print(ccdata['word_database']['re'])

['nobile', 'nobiltà', 'nobiliare', 'aristocratico', 'nobildonna', 'nobiluomo', 'casato', 'illustre', 'patrizio', 'casata', 'aristocrazia', 'nobilitato', 'umile', 'nobilitare', 'gentiluomo', 'benestante', 'discendente', 'ignobile', 'famiglia', 'eminente', 'amabile', 'conte', 'cortigiano', 'principe', 'possidente', 'stirpe', 'plebeo', 'lignaggio', 'magnatizio', 'marchese', 'gentildonna', 'nobilitazione', 'cavaliere', 'barone', 'contessa', 'duca', 'rampollo', 'antico', 'notabile', 'decurionale', 'virile', 'facoltoso', 'rispettabile', 'prosapia', 'baronale', 'signorotto', 'marchesana', 'lodevole', 'ceto', 'vile']


In [16]:
data = create_semantle_data(
    all_words=words,
    solution_words=common_words,
    model=wikimodel,
    date=datetime.date(2022,9,11),
    puzzle_number=1,
    verbose=True
)

La soluzione per il 2022-09-11 è "nobile"


In [28]:
data = create_semantle_data(
    all_words=words,
    solution_words=common_words,
    model=ccmodel,
    date=datetime.date(2022,9,11),
    puzzle_number=1,
    verbose=True
)
print(data["word_database"]["conte"])
print([n["w"] for n in data["closest_words_list"]])


La soluzione per il 2022-09-11 è "nobile"
{'s': 47.61, 'r': 26}
['nobile', 'nobiltà', 'nobiliare', 'nobili', 'nobilissimo', 'nobilitante', 'aristocratico', 'nobilissima', 'nobilitata', 'casato', 'illustre', 'patrizio', 'nobilmente', 'nobilitato', 'umile', 'antica', 'nobilitare', 'gentiluomo', 'benestante', 'discendente', 'nobilita', 'facoltosa', 'ignobile', 'famiglia', 'eminente', 'amabile', 'conte', 'principe', 'cortigiano', 'stirpe', 'plebeo', 'cavalleresca', 'gentildonna', 'marchese', 'cavaliere', 'barone', 'nobilitatore', 'nobilitino', 'contessa', 'duca', 'maritato', 'popolana', 'antico', 'notabile', 'antichissima', 'nobilitati', 'virile', 'facoltoso', 'agiata', 'prosapia', 'nobilitate', 'nobilitano', 'lodevole', 'raffinata', 'ceto', 'vile', 'fanciulla', 'feudatario', 'giovane', 'regnante', 'originaria', 'schiatta', 'caritatevole', 'prelato', 'nobiliti', 'rango', 'generoso', 'regale', 'potente', 'nobilitò', 'letterato', 'temperante', 'virtuosa', 'magnanimo', 'valoroso', 'volgare', 

In [17]:
for days_ahead in range(365):
    _date = datetime.date(2022,9,9)+datetime.timedelta(days=days_ahead)
    data = create_semantle_data(
        all_words=words,
        solution_words=common_words,
        model=ccmodel,
        date=_date,
        puzzle_number=days_ahead,
        verbose=True
    )
    isopath = _date.isoformat()
    os.makedirs("AppleCC/"+isopath, exist_ok = True)
    with open("AppleCC/"+isopath+"/semantle.json", "w") as file:
        json.dump(data,file)
    with open("AppleCC/"+isopath+"/closest.json", "w") as file:
        json.dump(data['closest_words_list'],file)
    with open("AppleCC/"+isopath+"/stats.json", "w") as file:
        json.dump(data['day_stats'],file)
    with open("AppleCC/"+isopath+"/database.json", "w") as file:
        json.dump(data['word_database'],file)

La soluzione per il 2022-09-09 è "zio"
La soluzione per il 2022-09-10 è "qualsiasi"
La soluzione per il 2022-09-11 è "nobile"
La soluzione per il 2022-09-12 è "segno"
La soluzione per il 2022-09-13 è "correre"
La soluzione per il 2022-09-14 è "insomma"
La soluzione per il 2022-09-15 è "caratteristico"
La soluzione per il 2022-09-16 è "accompagnare"
La soluzione per il 2022-09-17 è "settimana"
La soluzione per il 2022-09-18 è "rapporto"
La soluzione per il 2022-09-19 è "colore"
La soluzione per il 2022-09-20 è "regione"
La soluzione per il 2022-09-21 è "spesa"
La soluzione per il 2022-09-22 è "realtà"
La soluzione per il 2022-09-23 è "minuto"
La soluzione per il 2022-09-24 è "giornale"
La soluzione per il 2022-09-25 è "entro"
La soluzione per il 2022-09-26 è "necessità"
La soluzione per il 2022-09-27 è "viaggio"
La soluzione per il 2022-09-28 è "tuttavia"
La soluzione per il 2022-09-29 è "capitare"
La soluzione per il 2022-09-30 è "accorgersi"
La soluzione per il 2022-10-01 è "oggi"
La 

La soluzione per il 2023-03-19 è "muovere"
La soluzione per il 2023-03-20 è "mandare"
La soluzione per il 2023-03-21 è "periodo"
La soluzione per il 2023-03-22 è "vita"
La soluzione per il 2023-03-23 è "studiare"
La soluzione per il 2023-03-24 è "strano"
La soluzione per il 2023-03-25 è "leggero"
La soluzione per il 2023-03-26 è "sala"
La soluzione per il 2023-03-27 è "suo"
La soluzione per il 2023-03-28 è "giungere"
La soluzione per il 2023-03-29 è "limitare"
La soluzione per il 2023-03-30 è "mangiare"
La soluzione per il 2023-03-31 è "segreto"
La soluzione per il 2023-04-01 è "presso"
La soluzione per il 2023-04-02 è "figlio"
La soluzione per il 2023-04-03 è "diventare"
La soluzione per il 2023-04-04 è "movimento"
La soluzione per il 2023-04-05 è "principe"
La soluzione per il 2023-04-06 è "aiuto"
La soluzione per il 2023-04-07 è "eccellenza"
La soluzione per il 2023-04-08 è "sorprendere"
La soluzione per il 2023-04-09 è "ottenere"
La soluzione per il 2023-04-10 è "direttore"
La solu

In [76]:
wikidata.keys()

dict_keys(['word_database', 'closest_words_list', 'solution_word', 'day_stats'])